# Modeling & Evaluation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('ggplot')
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from scipy.stats import skew
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor

In [ ]:
X_train = pd.read_csv("dataset/X_train.csv", header=None)
X_test = pd.read_csv("dataset/X_test.csv", header=None)
y_train = pd.read_csv("dataset/y_train.csv", header=None)
y_train_log = np.log(y_train)
X_train.shape, y_train_log.shape

In [ ]:
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model,
                                    X, y, 
                                    scoring="neg_mean_squared_error", 
                                    cv=5))
    return rmse


In [ ]:
models = [LinearRegression(),
          Ridge(),Lasso(alpha=0.01,max_iter=10000),
          RandomForestRegressor(),GradientBoostingRegressor(),SVR(),
          LinearSVR(),ElasticNet(alpha=0.001,max_iter=10000),
          SGDRegressor(max_iter=1000,tol=1e-3),BayesianRidge(),
          KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5),
          ExtraTreesRegressor(),XGBRegressor()]

names = ["LR", "Ridge", "Lasso", "RF", "GBR", "SVR", 
         "LinSVR", "Ela","SGD","Bay","Ker","Extra","Xgb"]

for name, model in zip(names, models):
    score = rmse_cv(model, X_train, y_train_log)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

In [ ]:
class grid():
    def __init__(self,model):
        self.model = model
    
    def grid_get(self,X,y,param_grid):
        grid_search = GridSearchCV(self.model,param_grid,cv=5,
                                   scoring="neg_mean_squared_error")
        grid_search.fit(X,y)
        print(grid_search.best_params_, np.sqrt(-grid_search.best_score_))
        grid_search.cv_results_['mean_test_score'] = np.sqrt(
            -grid_search.cv_results_['mean_test_score'])
        print(pd.DataFrame(
            grid_search.cv_results_)[['params',
                                      'mean_test_score',
                                      'std_test_score']])

In [ ]:
grid(Lasso()).grid_get(X_train,y_train_log,
                       {'alpha': [0.0002,0.0004,0.0006,
                                  0.0008,0.0005,0.0007],
                        'max_iter':[10000]})

In [ ]:
grid(Ridge()).grid_get(
    X_train,y_train_log,{'alpha':range(62, 69)})

In [ ]:
grid(SVR()).grid_get(
    X_train,y_train_log,{'C':[8, 9, 10],
                    'kernel':["rbf"],
                    "gamma":[0.0004, 0.0005, 0.0006],
                    "epsilon":[0.009, 0.01]})

In [ ]:
param_grid={'alpha':[0.2,0.3,0.4,0.5], 'kernel':["polynomial"],
            'degree':[3],'coef0':[0.8,1,1.2]}
grid(KernelRidge()).grid_get(X_train,y_train_log,param_grid)

In [ ]:
grid(ElasticNet()).grid_get(
    X_train,y_train_log,{'alpha':[0.0008,0.004,0.005,0.006],
                         'l1_ratio':[0.08,0.1,0.3,0.5,0.7],
                         'max_iter':[10000]})

In [ ]:
class AverageWeight(BaseEstimator, RegressorMixin):
    def __init__(self,mod,weight):
        self.mod = mod
        self.weight = weight
        
    def fit(self,X,y):
        self.models_ = [clone(x) for x in self.mod]
        for model in self.models_:
            model.fit(X,y)
        return self
    
    def predict(self,X):
        w = list()
        pred = np.array([model.predict(X) for model in self.models_])
        for data in range(pred.shape[1]):
            single = [pred[model,data]*weight for model,weight in zip(range(pred.shape[0]),self.weight)]
            w.append(np.sum(single))
        return w

In [ ]:
lasso = Lasso(alpha=0.0006,max_iter=10000) # 0.10987
ridge = Ridge(alpha=66) # 0.10992
svr = SVR(gamma= 0.0004,kernel='rbf',C=9,epsilon=0.01) # 0.10786
ker = KernelRidge(alpha=0.4 ,kernel='polynomial',degree=3 , coef0=1.2) # 0.11423
ela = ElasticNet(alpha=0.005,l1_ratio=0.08,max_iter=10000) # 0.10983
bay = BayesianRidge() # 0.11020


In [ ]:
class stacking(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self,mod,meta_model):
        self.mod = mod
        self.meta_model = meta_model
        self.kf = KFold(n_splits=5, random_state=42, shuffle=True)
        
    def fit(self,X,y):
        self.saved_model = [list() for i in self.mod]
        oof_train = np.zeros((X.shape[0], len(self.mod)))
        
        for i,model in enumerate(self.mod):
            for train_index, val_index in self.kf.split(X,y):
                renew_model = clone(model)
                renew_model.fit(X[train_index], y[train_index])
                self.saved_model[i].append(renew_model)
                oof_train[val_index,i] = renew_model.predict(X[val_index])
        
        self.meta_model.fit(oof_train,y)
        return self
    
    def predict(self,X):
        whole_test = np.column_stack([np.column_stack(model.predict(X) for model in single_model).mean(axis=1) 
                                      for single_model in self.saved_model]) 
        return self.meta_model.predict(whole_test)
    
    def get_oof(self,X,y,test_X):
        oof = np.zeros((X.shape[0],len(self.mod)))
        test_single = np.zeros((test_X.shape[0],5))
        test_mean = np.zeros((test_X.shape[0],len(self.mod)))
        for i,model in enumerate(self.mod):
            for j, (train_index,val_index) in enumerate(self.kf.split(X,y)):
                clone_model = clone(model)
                clone_model.fit(X[train_index],y[train_index])
                oof[val_index,i] = clone_model.predict(X[val_index])
                test_single[:,j] = clone_model.predict(test_X)
            test_mean[:,i] = test_single.mean(axis=1)
        return oof, test_mean

In [ ]:
a = Imputer().fit_transform(X_train)
b = Imputer().fit_transform(y_train_log.values.reshape(-1,1)).ravel()

### Find best meta model

In [ ]:
metas = [lasso,ridge,svr,ker,ela,bay]
for meta in metas:
    stack_model = stacking(mod=[lasso,ridge,svr,ker,ela,bay],meta_model=meta)
    print(str(meta),rmse_cv(stack_model,a,b).mean())

In [ ]:
stack_model = stacking(mod=[lasso,ridge,svr,ker,ela,bay],meta_model=ker)


In [ ]:
X_train_stack, X_test_stack = stack_model.get_oof(a,b,X_test)

In [ ]:
X_train_add = np.hstack((a,X_train_stack))
X_test_add = np.hstack((X_test,X_test_stack))

In [ ]:
X_train_add.shape, X_test_add.shape

In [ ]:
print(rmse_cv(stack_model,X_train_add,b))
print(rmse_cv(stack_model,X_train_add,b).mean())

In [ ]:
stack_model = stacking(mod=[lasso,ridge,svr,ker,ela,bay],meta_model=ker)

In [ ]:
stack_model.fit(a,b)

In [ ]:
pred = np.exp(stack_model.predict(X_test))

In [ ]:
result=pd.DataFrame({'Id':range(1461, 2920), 'SalePrice':pred})
result.to_csv("dataset/submission.csv",index=False)